In [1]:
import pandas as pd

# path = 'D:/20240416/5. data_analysis_workspace/0_project/음원 데이터를 활용한 주간음원순위 예측.csv'

path = '음원예측.csv'
data = pd.read_csv(path)

#'rank', 'artist', 'name', 'week', 'title_song', 'st_day', 'rank_g_pred', 'rank_g']

In [7]:
data.columns

Index(['X1', 'artist', 'name', 'rank_g', 'title_song', 'week', 'runtime',
       'top_freq', 'gg_score', 'nv_score', 'total_view', 'season_genre_score',
       'pd_score', 'dc_total_numb', 'dc_mean_reccomend', 'dc_mean_views',
       'drama_view', 'st_day', 'rank', 'you_rank_g_1', 'you_rank_g_2',
       'you_rank_g_3', 'you_rank_g_4', 'you_rank_g_5', 'you_rank_g_6',
       'you_rank_g_7', 'you_rank_g_8', 'you_rank_g_9', 'you_rank_g_10',
       'genre_All', 'genre_Ani/Game', 'genre_Ballad', 'genre_Blues/Folk',
       'genre_Dance', 'genre_Drama', 'genre_Electronica',
       'genre_Foreign Movie', 'genre_Indi', 'genre_Jazz', 'genre_Pop',
       'genre_R&B/Soul', 'genre_Rap/Hiphop', 'genre_Rock', 'genre_Trot',
       'genre_carol', 'season_fall', 'season_spring', 'season_summer',
       'season_winter', 'sex_female', 'sex_male', 'sex_mixed',
       'song_type_K_POP', 'song_type_OST', 'song_type_Others', 'song_type_POP',
       'active_type_Band', 'active_type_Duet', 'active_type_Group',
 

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
import xgboost as xgb
from catboost import CatBoostRegressor
import lightgbm as lgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 데이터 로드
data = pd.read_csv('음원예측.csv')

# 제외할 컬럼 설정
exclude_columns = ['rank', 'artist', 'name', 'week', 'title_song', 'st_day', 'rank_g_pred', 'rank_g']  # 예시로 제외할 컬럼들

# 타겟을 'rank'로 설정하고 제외할 컬럼을 제거
X = data.drop(columns=['rank'] + exclude_columns)  # 제외할 컬럼들 및 target을 제외한 나머지 독립 변수
y = data['rank']  # target

# 학습/테스트 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 평가 지표 계산 함수
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    return rmse, r2

# 모델 저장용 딕셔너리
models = {
    'Linear Regression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'ElasticNet': ElasticNet(),
    'Random Forest': RandomForestRegressor(),
    'XGBoost': xgb.XGBRegressor(),
    'CatBoost': CatBoostRegressor(verbose=0),
    'LightGBM': lgb.LGBMRegressor(),
    'Gradient Descent': SGDRegressor(),
}

# 다항 회귀 모델 설정
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
poly_model = LinearRegression()

# 모델 학습 및 평가
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    rmse, r2 = evaluate_model(model, X_test, y_test)
    results[name] = {'RMSE': rmse, 'R²': r2}

# 다항 회귀 추가
poly_model.fit(X_train_poly, y_train)
rmse_poly, r2_poly = evaluate_model(poly_model, X_test_poly, y_test)
results['Polynomial Regression'] = {'RMSE': rmse_poly, 'R²': r2_poly}

# 딥러닝 모델 설정 및 학습
deep_model = Sequential()
deep_model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
deep_model.add(Dense(32, activation='relu'))
deep_model.add(Dense(1))

deep_model.compile(loss='mean_squared_error', optimizer='adam')
deep_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# 딥러닝 모델 평가
y_pred_deep = deep_model.predict(X_test)
rmse_deep = np.sqrt(mean_squared_error(y_test, y_pred_deep))
r2_deep = r2_score(y_test, y_pred_deep)
results['Deep Learning'] = {'RMSE': rmse_deep, 'R²': r2_deep}

# 결과 출력
results_df = pd.DataFrame(results).T
print(results_df)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000941 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2410
[LightGBM] [Info] Number of data points in the train set: 13650, number of used features: 75
[LightGBM] [Info] Start training from score 91.317436


C:\Users\user\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
                               RMSE            R²
Linear Regression      2.178755e+01  8.527769e-01
Lasso                  3.029526e+01  7.153517e-01
Ridge                  2.178861e+01  8.527625e-01
ElasticNet             4.970683e+01  2.337129e-01
Random Forest          1.905992e+01  8.873317e-01
XGBoost                1.891052e+01  8.890911e-01
CatBoost               1.818165e+01  8.974759e-01
LightGBM               1.831894e+01  8.959218e-01
Gradient Descent       1.224917e+16 -4.653426e+28
Polynomial Regression  2.968349e+01  7.267317e-01
Deep Learning          2.194560e+01  8.506331e-01


### 경사하강법 수정

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import SGDRegressor
import xgboost as xgb
from catboost import CatBoostRegressor
import lightgbm as lgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 데이터 로드
data = pd.read_csv('음원예측.csv')

# 제외할 컬럼 설정
exclude_columns = ['rank', 'artist', 'name', 'week', 'title_song', 'st_day', 'rank_g_pred', 'rank_g']

# 타겟을 'rank'로 설정하고 제외할 컬럼을 제거
X = data.drop(columns=['rank'] + exclude_columns)  # 제외할 컬럼들 및 target을 제외한 나머지 독립 변수
y = data['rank']  # target

# 학습/테스트 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 스케일링 (경사하강법에 필수)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 평가 지표 계산 함수
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    return rmse, r2

# 모델 저장용 딕셔너리
models = {
    'Linear Regression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'ElasticNet': ElasticNet(),
    'Random Forest': RandomForestRegressor(),
    'XGBoost': xgb.XGBRegressor(),
    'CatBoost': CatBoostRegressor(verbose=0),
    'LightGBM': lgb.LGBMRegressor(),
}

# 경사하강법 모델 추가 (학습률 조정)
sgd_regressor = SGDRegressor(alpha=0.01, max_iter=1000, tol=1e-3)

# 다항 회귀 모델 설정
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
poly_model = LinearRegression()

# 모델 학습 및 평가
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    rmse, r2 = evaluate_model(model, X_test, y_test)
    results[name] = {'RMSE': rmse, 'R²': r2}

# 경사하강법 회귀 모델 학습 및 평가 (스케일링된 데이터 사용)
sgd_regressor.fit(X_train_scaled, y_train)
rmse_sgd, r2_sgd = evaluate_model(sgd_regressor, X_test_scaled, y_test)
results['Gradient Descent'] = {'RMSE': rmse_sgd, 'R²': r2_sgd}

# 다항 회귀 추가
poly_model.fit(X_train_poly, y_train)
rmse_poly, r2_poly = evaluate_model(poly_model, X_test_poly, y_test)
results['Polynomial Regression'] = {'RMSE': rmse_poly, 'R²': r2_poly}

# 딥러닝 모델 설정 및 학습
deep_model = Sequential()
deep_model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
deep_model.add(Dense(32, activation='relu'))
deep_model.add(Dense(1))

deep_model.compile(loss='mean_squared_error', optimizer='adam')
deep_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# 딥러닝 모델 평가
y_pred_deep = deep_model.predict(X_test)
rmse_deep = np.sqrt(mean_squared_error(y_test, y_pred_deep))
r2_deep = r2_score(y_test, y_pred_deep)
results['Deep Learning'] = {'RMSE': rmse_deep, 'R²': r2_deep}

# 결과 출력
results_df = pd.DataFrame(results).T
print(results_df)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001024 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2410
[LightGBM] [Info] Number of data points in the train set: 13650, number of used features: 75
[LightGBM] [Info] Start training from score 91.317436


C:\Users\user\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
                                RMSE            R²
Linear Regression          21.787546  8.527769e-01
Lasso                      30.295258  7.153517e-01
Ridge                      21.788611  8.527625e-01
ElasticNet                 49.706831  2.337129e-01
Random Forest              19.120841  8.866104e-01
XGBoost                    18.910521  8.890911e-01
CatBoost                   18.181655  8.974759e-01
LightGBM                   18.318937  8.959218e-01
Gradient Descent       111469.375843 -3.853633e+06
Polynomial Regression      29.683489  7.267317e-01
Deep Learning              24.749006  8.100345e-01


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.linear_model import HuberRegressor, BayesianRidge, PassiveAggressiveRegressor
from sklearn.preprocessing import StandardScaler

# 데이터 로드
data = pd.read_csv('음원예측.csv')

# 제외할 컬럼 설정
exclude_columns = ['rank', 'artist', 'name', 'week', 'title_song', 'st_day', 'rank_g_pred', 'rank_g']

# 타겟을 'rank'로 설정하고 제외할 컬럼을 제거
X = data.drop(columns=['rank'] + exclude_columns)  # 제외할 컬럼들 및 target을 제외한 나머지 독립 변수
y = data['rank']  # target

# 학습/테스트 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 스케일링 (필요한 모델에 적용)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 평가 지표 계산 함수
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    return rmse, r2

# 새로운 모델 저장용 딕셔너리
new_models = {
    'K-Nearest Neighbors': KNeighborsRegressor(),
    'Support Vector Regressor (SVR)': SVR(),
    'Decision Tree': DecisionTreeRegressor(),
    'Bagging Regressor': BaggingRegressor(),
    'AdaBoost Regressor': AdaBoostRegressor(),
    'Gradient Boosting Regressor': GradientBoostingRegressor(),
    'Extra Trees Regressor': ExtraTreesRegressor(),
    'Huber Regressor': HuberRegressor(),
    'Bayesian Ridge': BayesianRidge(),
    'Passive Aggressive Regressor': PassiveAggressiveRegressor(max_iter=1000, tol=1e-3),
}

# 모델 학습 및 평가
results = {}
for name, model in new_models.items():
    if 'Huber' in name or 'Passive Aggressive' in name or 'SVR' in name or 'K-Nearest Neighbors' in name:
        # 스케일링된 데이터를 사용하는 모델들
        model.fit(X_train_scaled, y_train)
        rmse, r2 = evaluate_model(model, X_test_scaled, y_test)
    else:
        # 스케일링이 필요 없는 모델들
        model.fit(X_train, y_train)
        rmse, r2 = evaluate_model(model, X_test, y_test)
    results[name] = {'RMSE': rmse, 'R²': r2}

# 결과 출력
results_df = pd.DataFrame(results).T
print(results_df)



                                     RMSE        R²
K-Nearest Neighbors             23.446487  0.829504
Support Vector Regressor (SVR)  26.304024  0.785413
Decision Tree                   24.923252  0.807350
Bagging Regressor               19.608583  0.880752
AdaBoost Regressor              45.028770  0.371161
Gradient Boosting Regressor     21.141570  0.861377
Extra Trees Regressor           19.637741  0.880397
Huber Regressor                 22.295671  0.845830
Bayesian Ridge                  21.787967  0.852771
Passive Aggressive Regressor    28.443875  0.749079


In [12]:
import pandas as pd

# path = 'D:/20240416/5. data_analysis_workspace/0_project/음원 데이터를 활용한 주간음원순위 예측.csv'

path = 'circle_data.csv'
data = pd.read_csv(path)

In [21]:

data['Genre'].drop_duplicates().reset_index()

,index,Genre
0,0,가요 / 댄스
1,1,가요 / 블루스/포크
2,2,가요 / 랩/힙합
3,7,가요 / 발라드
4,13,가요 / R&B/소울
5,16,OST / 전체
6,22,가요 / 락
7,35,OST / 드라마
8,39,가요 / 인디
9,41,POP / 팝


In [31]:
# data['Genre'][data['Genre'].isna()]


data[data['Genre'].isna()]

,Unnamed: 0,date,Rank,Title,Artist,Production,Weekly Views,Genre,Runtime,New_Artist,ky_rank
2555,2555,2023-12-17,156,넌 쉽게 말했지만,싱어게인3 49호 가수,"SLL, 뮤직팜",NaN,NaN,NaN,싱어게인3 49호 가수,NaN
2985,2985,2023-12-31,186,넌 쉽게 말했지만,싱어게인3 49호 가수,"SLL, 뮤직팜",NaN,NaN,NaN,싱어게인3 49호 가수,NaN
3144,3144,2024-01-07,145,넌 쉽게 말했지만,싱어게인3 49호 가수,"SLL, 뮤직팜",NaN,NaN,NaN,싱어게인3 49호 가수,NaN
3340,3340,2024-01-14,141,넌 쉽게 말했지만,싱어게인3 49호 가수,"SLL, 뮤직팜",NaN,NaN,NaN,싱어게인3 49호 가수,NaN
3568,3568,2024-01-21,169,넌 쉽게 말했지만,싱어게인3 49호 가수,"SLL, 뮤직팜",NaN,NaN,NaN,싱어게인3 49호 가수,NaN
